# 🧹 Limpieza y Preparación de Datos

**Objetivo:** Limpiar y preparar los datos para el modelo de recomendación

**Fecha:** 22 de noviembre de 2024

## Tareas:
1. Cargar datos
2. Identificar y manejar valores faltantes
3. Eliminar duplicados
4. Filtrar usuarios y películas con pocas calificaciones
5. Guardar datos limpios

In [2]:
import pandas as pd
import numpy as np

# Cargar datos
columns = ['user_id', 'item_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/raw/ml-100k/u.data', 
                      sep='\t', 
                      names=columns,
                      encoding='latin-1')

movie_columns = ['item_id', 'title', 'release_date', 'video_release_date',
                 'imdb_url', 'unknown', 'Action', 'Adventure', 'Animation',
                 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 
                 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('../data/raw/ml-100k/u.item', 
                     sep='|', 
                     names=movie_columns,
                     encoding='latin-1')

print(f"✅ Datos cargados")
print(f"📊 Ratings originales: {len(ratings):,}")
print(f"🎬 Películas originales: {len(movies):,}")

✅ Datos cargados
📊 Ratings originales: 100,000
🎬 Películas originales: 1,682


In [3]:
print("🔍 VERIFICACIÓN DE CALIDAD DE DATOS\n")

# Valores faltantes en ratings
print("📊 Valores faltantes en ratings:")
print(ratings.isnull().sum())
print()

# Duplicados
print(f"🔄 Duplicados en ratings: {ratings.duplicated().sum()}")
print()

# Rango de calificaciones
print(f"⭐ Rango de calificaciones: {ratings['rating'].min()} - {ratings['rating'].max()}")
print()

# Verificar IDs
print(f"👥 Usuarios únicos: {ratings['user_id'].nunique()}")
print(f"🎬 Películas únicas: {ratings['item_id'].nunique()}")

🔍 VERIFICACIÓN DE CALIDAD DE DATOS

📊 Valores faltantes en ratings:
user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

🔄 Duplicados en ratings: 0

⭐ Rango de calificaciones: 1 - 5

👥 Usuarios únicos: 943
🎬 Películas únicas: 1682


In [4]:
# Eliminar duplicados
ratings_clean = ratings.drop_duplicates()

print(f"🗑️ Duplicados eliminados: {len(ratings) - len(ratings_clean)}")
print(f"✅ Ratings después de limpieza: {len(ratings_clean):,}")

🗑️ Duplicados eliminados: 0
✅ Ratings después de limpieza: 100,000


In [5]:
# Contar calificaciones por usuario
user_counts = ratings_clean.groupby('user_id').size()

# Filtrar usuarios con al menos 20 calificaciones
min_user_ratings = 20
active_users = user_counts[user_counts >= min_user_ratings].index

ratings_filtered = ratings_clean[ratings_clean['user_id'].isin(active_users)]

print(f"👥 Usuarios con menos de {min_user_ratings} ratings eliminados")
print(f"   Usuarios originales: {ratings_clean['user_id'].nunique()}")
print(f"   Usuarios activos: {len(active_users)}")
print(f"   Usuarios eliminados: {ratings_clean['user_id'].nunique() - len(active_users)}")
print(f"📊 Ratings restantes: {len(ratings_filtered):,}")

👥 Usuarios con menos de 20 ratings eliminados
   Usuarios originales: 943
   Usuarios activos: 943
   Usuarios eliminados: 0
📊 Ratings restantes: 100,000


In [6]:
# Contar calificaciones por película
movie_counts = ratings_filtered.groupby('item_id').size()

# Filtrar películas con al menos 20 calificaciones
min_movie_ratings = 20
popular_movies = movie_counts[movie_counts >= min_movie_ratings].index

ratings_final = ratings_filtered[ratings_filtered['item_id'].isin(popular_movies)]

print(f"🎬 Películas con menos de {min_movie_ratings} ratings eliminadas")
print(f"   Películas originales: {ratings_filtered['item_id'].nunique()}")
print(f"   Películas populares: {len(popular_movies)}")
print(f"   Películas eliminadas: {ratings_filtered['item_id'].nunique() - len(popular_movies)}")
print(f"📊 Ratings finales: {len(ratings_final):,}")

🎬 Películas con menos de 20 ratings eliminadas
   Películas originales: 1682
   Películas populares: 939
   Películas eliminadas: 743
📊 Ratings finales: 94,968


In [7]:
print("=" * 60)
print("📊 RESUMEN DE LIMPIEZA DE DATOS")
print("=" * 60)
print()
print(f"📥 Datos originales:")
print(f"   • Ratings: {len(ratings):,}")
print(f"   • Usuarios: {ratings['user_id'].nunique()}")
print(f"   • Películas: {ratings['item_id'].nunique()}")
print()
print(f"📤 Datos limpios:")
print(f"   • Ratings: {len(ratings_final):,}")
print(f"   • Usuarios: {ratings_final['user_id'].nunique()}")
print(f"   • Películas: {ratings_final['item_id'].nunique()}")
print()
print(f"🗑️ Datos eliminados:")
print(f"   • Ratings: {len(ratings) - len(ratings_final):,} ({(1 - len(ratings_final)/len(ratings))*100:.1f}%)")
print(f"   • Usuarios: {ratings['user_id'].nunique() - ratings_final['user_id'].nunique()}")
print(f"   • Películas: {ratings['item_id'].nunique() - ratings_final['item_id'].nunique()}")
print()
print("=" * 60)

📊 RESUMEN DE LIMPIEZA DE DATOS

📥 Datos originales:
   • Ratings: 100,000
   • Usuarios: 943
   • Películas: 1682

📤 Datos limpios:
   • Ratings: 94,968
   • Usuarios: 943
   • Películas: 939

🗑️ Datos eliminados:
   • Ratings: 5,032 (5.0%)
   • Usuarios: 0
   • Películas: 743



In [9]:
# Guardar datos procesados
ratings_final.to_csv('../data/processed/ratings_clean.csv', index=False)

# Guardar también la lista de películas válidas
movies_clean = movies[movies['item_id'].isin(ratings_final['item_id'].unique())]
movies_clean.to_csv('../data/processed/movies_clean.csv', index=False)

print("✅ Datos limpios guardados en:")
print("   📁 data/processed/ratings_clean.csv")
print("   📁 data/processed/movies_clean.csv")
print()
print(f"💾 Tamaño del archivo de ratings: {len(ratings_final):,} filas")
print(f"💾 Tamaño del archivo de películas: {len(movies_clean):,} filas")

✅ Datos limpios guardados en:
   📁 data/processed/ratings_clean.csv
   📁 data/processed/movies_clean.csv

💾 Tamaño del archivo de ratings: 94,968 filas
💾 Tamaño del archivo de películas: 939 filas


In [10]:
# Verificar que se guardaron correctamente
test_ratings = pd.read_csv('../data/processed/ratings_clean.csv')
test_movies = pd.read_csv('../data/processed/movies_clean.csv')

print("🔍 VERIFICACIÓN DE ARCHIVOS GUARDADOS\n")
print(f"✅ Ratings cargados: {len(test_ratings):,} filas")
print(f"✅ Películas cargadas: {len(test_movies):,} filas")
print()
print("🎉 ¡Limpieza de datos completada exitosamente!")

🔍 VERIFICACIÓN DE ARCHIVOS GUARDADOS

✅ Ratings cargados: 94,968 filas
✅ Películas cargadas: 939 filas

🎉 ¡Limpieza de datos completada exitosamente!
